# Pre-Processing

Importing Necessary libraries

In [ ]:
import urllib.request,json
import sys
import os

bPath="../data/xeno-canto-dataset-full-all-Countries/"  #path of the dataset available from xeno canto website

In [ ]:
def save_json(searchTerms, birdName, country):
    numPages = 1
    page = 1
    # create a path to save json files and recordings
    path = bPath + birdName.replace(':', '') + "/" + country
    if not os.path.exists(path):
        print("Creating subdirectory " + path + " for downloaded files...")
        os.makedirs(path)
        # download a json file for every page found in a query
    while page < numPages + 1:
        print("Loading page " + str(page) + "...")
        url = 'https://www.xeno-canto.org/api/2/recordings?query={0}&page={1}'.format(searchTerms.replace(' ', '%20'),
                                                                                      page)
        print("url",url)
        jsonPage = urllib.request.urlopen(url)
        jsondata = json.loads(jsonPage.read().decode('utf-8'))
        filename = path + "/jsondata_p" + str(page) + ".json"
        with open(filename, 'w') as outfile:
            json.dump(jsondata, outfile)
        # check number of pages
        numPages = jsondata['numPages']
        page = page + 1
    print("Found ", numPages, " pages in total.")
    # return number of files in json
    # each page contains 500 results, the last page can have less than 500 records
    print("Saved json for ", (numPages - 1) * 500 + len(jsondata['recordings']), " files")
    return path

# reads the json and return the list of values for selected json part
# i.e. "id" - ID number, "type": type of the bird sound such as call or song
# for all Xeno Canto files found with the given search terms.
def read_data(searchTerm, path):
    data = []
    numPages = 1
    page = 1
    # read all pages and save results in a list
    while page < numPages + 1:
        # read file
        with open(path + "/jsondata_p" + str(page) + ".json", 'r') as jsonfile:
            jsondata = jsonfile.read()
        jsondata = json.loads(jsondata)
        # check number of pages
        numPages = jsondata['numPages']
        # find "recordings" in a json and save a list with a search term
        for k in range(len(jsondata['recordings'])):
            data.append(jsondata["recordings"][k][searchTerm])
        page = page + 1
    return data


# downloads all sound files found with the search terms into xeno-canto directory
# into catalogue named after the search term (i.e. Apus apus)
# filename have two parts: the name of the bird in latin and ID number
def download(searchTerms, birdName, country):
    # create data/xeno-canto-dataset directory
    print("X",searchTerms, birdName, country)
    path = save_json(searchTerms, birdName, country)
    print("path",path)
    # get filenames: recording ID and bird name in latin from json
    filenamesID = read_data('id', path)
    filenamesCountry = read_data('cnt', path)
    print(filenamesID,filenamesCountry)
    # get website recording http download address from json
    fileaddress = read_data('file', path)
    numfiles = len(filenamesID)
    print("A total of ", numfiles, " files will be downloaded")
    for i in range(0, numfiles):
        print("Saving file ", i + 1, "/", numfiles,
              bPath + birdName.replace(':', '') + filenamesID[
                  i] + ".mp3")
        urllib.request.urlretrieve("http:" + fileaddress[i],
                                   path + "/" + birdName + filenamesID[i] + ".mp3")

Downloading the audios of the given birds of the given country

In [ ]:
countries = ['Poland', 'Germany', 'Slovakia', 'Czech', 'Lithuania']
birds = ['Coccothraustescoccothraustes',
        'Columbapalumbus',
        'Delichonurbicum',
        'Apusapus',
        'Sittaeuropaea',
        'Phoenicurusochruros']

for country in countries:
    for bird in birds:
        download(bird + ' cnt:' + country + ' type:song', bird.replace(' ', ''), country)
for bird in birds:
    download(bird + ' type:song', bird.replace(' ', ''), 'countries')

mp3 to wav Conversion

In [ ]:
import os
from pydub import AudioSegment
birds = ['Coccothraustescoccothraustes',
        'Columbapalumbus',
        'Delichonurbicum',
        'Apusapus',
        'Sittaeuropaea',
        'Phoenicurusochruros']

for b in birds:
    path = "C:/Users/data/xeno-canto-dataset-full-all-Countries/{0}/countries".format(b)

    #Change working directory
    os.chdir(path)

    audio_files = os.listdir()

    # You dont need the number of files in the folder, just iterate over them directly using:
    for file in audio_files:
        #spliting the file into the name and the extension
        name, ext = os.path.splitext(file)
        if ext == ".mp3":
           mp3_sound = AudioSegment.from_mp3(file)
           #rename them using the old name + ".wav"
           mp3_sound.export("{0}.wav".format(name), format="wav")

Making of the csv file by feature extraction from the wav file using the librosa library, this csv file will be used as
the data set for ANN and SVM models.

In [ ]:
file = open('bird dataset.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
birds = ['Coccothraustescoccothraustes',
        'Columbapalumbus',
        'Delichonurbicum',
        'Apusapus',
        'Sittaeuropaea',
        'Phoenicurusochruros']
for b in birds:
    for filename in os.listdir('F:\\data\\xeno-canto-dataset-full-all-Countries\\{0}\\countries_wav'.format(b)):
        birdsong = f'F:\\data\\xeno-canto-dataset-full-all-Countries\\{b}\\countries_wav\\{filename}'
        #extracting of different features from the wav file
        y, sr = librosa.load(birdsong, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {b}'
        file = open('bird dataset.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

wav to png Conversion for CNN model

In [ ]:
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
birds = ['Coccothraustescoccothraustes',
        'Columbapalumbus',
        'Delichonurbicum',
        'Apusapus',
        'Sittaeuropaea',
        'Phoenicurusochruros']

for b in birds:
    pathlib.Path(f'img_data/{b}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'F:\\data\\xeno-canto-dataset-full-all-Countries\\{b}\\countries_wav'):
        birdsound = f'F:\\data\\xeno-canto-dataset-full-all-Countries\\{b}\\countries_wav\\{filename}'
        y, sr = librosa.load(birdsound, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{b}/{filename[:-3].replace(".", "")}.png')
        plt.clf()